## Setup

In [ ]:
# linear vs. BDT selection performance with data/MC comparisons
# make sure to update the plots_path here & in backend function scripts 

In [ ]:
import sys
import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT


In [ ]:
from datetime import datetime
import time
now = datetime.now()
date_time = now.strftime("%H:%M:%S")
print("date and time:",date_time)

In [ ]:
import NuMIGeoWeights
importlib.reload(NuMIGeoWeights)

# the default option is FHC, RHC needs different arguments
numiBeamlineGeoWeights = NuMIGeoWeights.NuMIGeoWeights() 

In [ ]:
importlib.reload(sf)
from selection_functions import *

In [ ]:
# Doing Run3??
ISRUN3 = False

In [ ]:
# use nue intrinsic? 
NUE_INTRINSIC = True

In [ ]:
if ISRUN3: 
    plots_path = "/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/rhc/"

else: 
    plots_path = "/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/"


In [ ]:
# POT normalization factors

# FHC
if not ISRUN3:  
    overlay_pot =  2.33652E21  # v7       
    dirt_pot = 1.67392E21 # david's file
    beamon_pot = 2.0E20 # v5

    beamon_ntrig =  5268051.0 # v5 (EA9CNT_wcut)
    beamoff_ntrig = 9199232.74  # v5 (EXT_NUMIwin_FEMBeamTriggerAlgo)
    
    if NUE_INTRINSIC: 
        nue_intrinsic_pot = 2.37838E22 # v7
    

# RHC 
else: 
    overlay_pot =  1.98937E21 # v7
    dirt_pot = 4.65831E20 # v3
    beamon_pot = 5.0E20 # v5
    
    beamon_ntrig = 10363728.0 # v5
    beamoff_ntrig =  32878305.25 # v5
        
    if NUE_INTRINSIC: 
        nue_intrinsic_pot = 2.5345E22 # v7
    


In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

DATA = ""
EXT = ""
OVRLY  = ""
DRT = ""
NUE = ""

#path = '/uboone/data/users/kmistry/work/MCC9/searchingfornues/'

#reduced
#path = '/uboone/app/users/kmiller/searchingfornues_v33/Jan2021_NuMI_workshopDay2/Day1/root_files/'

# reduced with opening angle 
path = '/uboone/data/users/kmiller/ntuples/run1/nuepresel/'

if not ISRUN3: 
    
    # Run 1 FHC 
    OVRLY = 'neutrinoselection_filt_run1_overlay_v7'
    EXT = 'neutrinoselection_filt_run1_beamoff_v5'
    DATA = 'neutrinoselection_filt_run1_beamon_beamgood_v5'
    DRT = 'prodgenie_numi_uboone_overlay_dirt_fhc_mcc9_run1_v28_all_snapshot'
    
    if NUE_INTRINSIC: 
        NUE = 'neutrinoselection_filt_run1_overlay_intrinsic_v7'

else: 
    # Run 3 RHC
    OVRLY = 'ntuple_files_v7/neutrinoselection_filt_run3b_overlay'
    DATA = 'ntuple_files_v5/neutrinoselection_filt_run3b_beamon_beamgood'
    EXT = 'ntuple_files_v5/neutrinoselection_filt_run3b_beamoff'
    DRT = 'ntuple_files_v3/neutrinoselection_filt_run3b_dirt_overlay'
    
    if NUE_INTRINSIC: 
        NUE = 'ntuple_files_v7/neutrinoselection_filt_run3b_overlay_intrinsic'



In [ ]:
overlay = uproot.open(path+OVRLY+".root")[fold][tree]
data = uproot.open(path+DATA+".root")[fold][tree]
ext = uproot.open(path+EXT+".root")[fold][tree]
dirt = uproot.open(path+DRT+".root")[fold][tree]  

uproot_v = [overlay,data,ext,dirt]

if NUE_INTRINSIC: 
    nue = uproot.open(path+NUE+".root")[fold][tree]
    uproot_v.append(nue)


In [ ]:
variables = [
    "selected", "nu_pdg", "shr_theta", "true_e_visible", 
    "trk_score_v", 
    "shr_tkfit_dedx_Y", "ccnc", "n_tracks_contained", 
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trkshrhitdist2",
    "nproton", "nu_e", "n_showers_contained", "nu_purity_from_pfp", 
    "shr_phi", "trk_phi", "trk_theta",
    "shr_score", 
    "trk_energy", "tksh_distance", "tksh_angle",
    "npi0",
    "shr_energy_tot_cali",  
    "nslice", 
    "contained_fraction",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "npion", "shr_energy_cali", 
    "shrmoliereavg", "shr_px", "shr_py", "shr_pz"
]


In [ ]:

# MC only variables
mc_var = ["weightSplineTimesTune", "true_nu_px", "true_nu_py", "true_nu_pz", 
            "elec_e", "proton_e", "mc_px", "mc_py", "mc_pz", "elec_px", "elec_py", "elec_pz", 
            "swtrig_pre", "ppfx_cv", "mc_pdg", "opening_angle"]

sys_genie = ["weightsGenie", "weightsReint"]
sys_flux = ['weightsPPFX']


## Create pandas dataframes

In [ ]:
overlay = overlay.pandas.df(variables + mc_var + sys_genie + sys_flux, flatten=False)

In [ ]:
overlay['weightsPPFX'] = overlay['weightsPPFX']/1000

In [ ]:
dirt = dirt.pandas.df(variables + mc_var + sys_genie, flatten=False)

In [ ]:
# no flux weights exist for dirt (yet?)
dirt['weightsPPFX'] = [[1 for x in range(len(overlay['weightsPPFX'].iloc[0]))] for y in range(len(dirt))]

In [ ]:
if NUE_INTRINSIC: 
    nue = nue.pandas.df(variables + mc_var + sys_genie + sys_flux, flatten=False)
    nue['weightsPPFX'] = nue['weightsPPFX']/1000

In [ ]:
data = data.pandas.df(variables, flatten=False) 

In [ ]:
ext = ext.pandas.df(variables, flatten=False)

In [ ]:
# not all datasets have all the variables
# but we want them to be of equal # of columns (and in order, annoyingly)

for var in mc_var+sys_genie+sys_flux: 
    data[var] = np.nan
    ext[var] = np.nan

In [ ]:
# how to get the LLR-PID value for the "track candidate" 
# (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!
#LLR-PID : log likelihood ratio particle ID 

df_v = [overlay,data,ext,dirt]

if NUE_INTRINSIC: 
    df_v.append(nue)
    
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    #df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['NeutrinoEnergy2_GeV'] = df['NeutrinoEnergy2']/1000


In [ ]:
# Add truth level theta & phi angles (detector & beam coordinates)
overlay = addAngles(overlay)

In [ ]:
dirt = addAngles(dirt)

In [ ]:
if NUE_INTRINSIC: 
    nue = addAngles(nue)

In [ ]:
mc_df = [overlay, dirt]

if NUE_INTRINSIC: 
    mc_df.append(nue)

In [ ]:
# add beamline geometry weights 

for i,df in enumerate(mc_df):
    df['weightsNuMIGeo'] = df.apply( lambda x: numiBeamlineGeoWeights.calculateGeoWeight(x['nu_pdg'],x['nu_e'],x['thbeam']) , axis=1)
    

In [ ]:
# not all datasets have all the variables, but we want them to be of equal # of columns

nan_var = ['thdet', 'phidet', 'true_nu_px_beam', 'true_nu_py_beam', 'true_nu_pz_beam', 'thbeam', 'phibeam','weightsNuMIGeo']

for var in nan_var: 
    data[var] = np.nan
    ext[var] = np.nan

In [ ]:
#np.setdiff1d(ext.columns,overlay.columns)
#ext.columns == overlay.columns

## Weights

In [ ]:
# cuts applied for bad GENIE weights 
for i,df in enumerate(mc_df):
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

In [ ]:
# pot scaling weights 

# applied tunes 
dirt_tune = 1
ext_tune = 1

if not ISRUN3: 
    dirt_tune = 0.35
    ext_tune = .98
    
else: 
    dirt_tune = 0.35
    ext_tune = .94
    
    
##############################################
# SCALE  TO BEAM ON POT
overlay_scale = beamon_pot/overlay_pot

if NUE_INTRINSIC: 
    nue_scale = beamon_pot/nue_intrinsic_pot

dirt_scale = dirt_tune*(beamon_pot/dirt_pot)
beamoff_scale = ext_tune*(beamon_ntrig/beamoff_ntrig) # scale factor to beam on POT

overlay['pot_scale'] = overlay_scale 

if NUE_INTRINSIC: 
    nue['pot_scale'] = nue_scale
    
dirt['pot_scale'] = dirt_scale
ext['pot_scale'] = beamoff_scale
data['pot_scale'] = [1 for x in range(len(data))]
##############################################
# SCALE TO OVERLAY

dirt_scale2 = dirt_tune*(overlay_pot/dirt_pot)
beamoff_scale2 = ext_tune*((overlay_pot/beamon_pot)*(beamon_ntrig/beamoff_ntrig))

if NUE_INTRINSIC: 
    nue_scale2 = overlay_pot/nue_intrinsic_pot

overlay['pot_scale_overlay'] = [1 for x in range(len(overlay))]
if NUE_INTRINSIC: 
    nue['pot_scale_overlay'] = nue_scale2
    
dirt['pot_scale_overlay'] = dirt_scale2
ext['pot_scale_overlay'] = beamoff_scale2
data['pot_scale_overlay'] = [1 for x in range(len(data))]
##############################################
# SCALE TO PROJECTED 
proj_pot = 0.0

if not ISRUN3: 
    proj_pot = 9.23E20 # FHC
else: 
    proj_pot = 11.95E20 # RHC

overlay_scale3 = proj_pot/overlay_pot
dirt_scale3 = dirt_tune*(proj_pot/dirt_pot)

if NUE_INTRINSIC: 
    nue_scale3 = proj_pot/nue_intrinsic_pot

# first scale to beamon, then scale to projected
beamoff_scale3 = (ext_tune*(beamon_ntrig/beamoff_ntrig)) * (proj_pot/beamon_pot)

overlay['pot_scale_proj'] = overlay_scale3

if NUE_INTRINSIC: 
    nue['pot_scale_proj'] = nue_scale3
    
dirt['pot_scale_proj'] = dirt_scale3
ext['pot_scale_proj'] = beamoff_scale3
data['pot_scale_proj'] = [1 for x in range(len(data))]

    
    

In [ ]:
# total weights 

# combined genie * POT weight * flux weight 
# ext gets POT weight only 

flux_weight = 'ppfx_cv'

################################################################
# totweight scales to BEAMON

# tuned
overlay['totweight'] = overlay['pot_scale']*overlay[flux_weight]*overlay['weightSplineTimesTune']
dirt['totweight'] = dirt['pot_scale']*dirt[flux_weight]*dirt['weightSplineTimesTune']

if NUE_INTRINSIC: 
    nue['totweight'] = nue['pot_scale']*nue[flux_weight]*nue['weightSplineTimesTune']


################################################################
# totweight_overlay scales to STANDARD OVERLAY

# tuned
overlay['totweight_overlay'] = overlay[flux_weight]*overlay['weightSplineTimesTune']
dirt['totweight_overlay'] = dirt['pot_scale_overlay']*dirt[flux_weight]*dirt['weightSplineTimesTune']

if NUE_INTRINSIC:
    nue['totweight_overlay'] = nue['pot_scale_overlay']*nue[flux_weight]*nue['weightSplineTimesTune']

################################################################
# totweight_proj scales to TOTAL PROJECTED BEAM ON 

overlay['totweight_proj'] = overlay['pot_scale_proj']*overlay[flux_weight]*overlay['weightSplineTimesTune']
dirt['totweight_proj'] = dirt['pot_scale_proj']*dirt[flux_weight]*dirt['weightSplineTimesTune']

if NUE_INTRINSIC:
    nue['totweight_proj'] = nue['pot_scale_proj']*nue[flux_weight]*nue['weightSplineTimesTune']


In [ ]:
# add offline flux weights? 
if offlineFluxWeights: 
    mc_df = offline_flux_weights(mc_df)

In [ ]:
# to keep the number of columns the same 
new_var = ['totweight', 'totweight_overlay', 'totweight_proj']

if offlineFluxWeights: 
    new_var.append('weightFlux')

for var in new_var: 
    for df in [data, ext]: 
        df[var] = np.nan

## compare intrinsic with standard overlay events

In [ ]:
in_fv_query = "10<=true_nu_vtx_x<=246 and -106<=true_nu_vtx_y<=106 and 10<=true_nu_vtx_z<=1026"
reco_in_fv_query = "10<=reco_nu_vtx_sce_x<=246 and -106<=reco_nu_vtx_sce_y<=106 and 10<=reco_nu_vtx_sce_z<=1026"

nueCC_query = 'abs(nu_pdg)==12 and ccnc==0 and '+in_fv_query

In [ ]:
nueCC_overlay = overlay.query(nueCC_query)

In [ ]:
# apply FV cut to intrinsic (already applied to overlay)
nue_fv = nue.query(in_fv_query)

In [ ]:
len(nueCC_overlay)

In [ ]:
len(nue_fv)

In [ ]:
#  with latest genie tune 

weight = 'totweight_overlay'

# stat error bars on standard overlay 
n, b, p = plt.hist(nueCC_overlay.query(SEL_QUERY)['nu_e'], 25, range=[0, 5], 
                   weights=nueCC_overlay.query(SEL_QUERY)[weight])
plt.close()

bincenters = 0.5*(b[1:]+b[:-1])
overlay_err =  np.sqrt(n)

fig = plt.figure(figsize=(10, 7))

plt.hist(nue_fv.query(SEL_QUERY)['nu_e'], 25, range=[0, 5], 
         weights=nue_fv.query(SEL_QUERY)[weight], alpha=0.7, label='nue intrinsic')

plt.hist(nueCC_overlay.query(SEL_QUERY)['nu_e'], 25, range=[0, 5], 
         weights=nueCC_overlay.query(SEL_QUERY)[weight], alpha=0.7, 
         label='standard overlay')

plt.errorbar(bincenters, n, yerr=overlay_err, fmt='none', color='black', linewidth=1)

plt.xlabel('true neutrino energy [GeV]', fontsize=15)
plt.ylabel('events (normalized to 2.3E21 POT)', fontsize=15)
plt.legend(fontsize=15)
#plt.ylim(0, 1300)
plt.title('selected nue/nuebar CC events, unweighted', fontsize=15)
plt.grid()
#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/intrinsic_compare_weighted_selected.pdf', 
#           transparent=True, bbox_inches='tight')
plt.show()

## Categories

In [ ]:
if NUE_INTRINSIC: 
    # intrinsic sample contains in AV TPC events ONLY, & only CC events (overlay is entire cryo)
    in_AV_query = "-1.55<=true_nu_vtx_x<=254.8 and -116.5<=true_nu_vtx_y<=116.5 and 0<=true_nu_vtx_z<=1036.8"
    
    # remove the nue/nuebar CC in overlay
    nueCC_query = 'abs(nu_pdg)==12 and ccnc==0 and '+in_AV_query
    print("# of nueCC in AV in overlay sample = "+str(len(overlay.query(nueCC_query))))
    len1 = len(overlay)
    
    idx = overlay.query(nueCC_query).index
    overlay.drop(idx, inplace=True)
    len2 = len(overlay) 
    print("# of nueCC in AV dropped in overlay = "+str(len1-len2))
    
    # then add in nue_intrinsic 
    overlay = pd.concat([overlay,nue], ignore_index=True)

    # from here on out everything else should be the same. 


In [ ]:
# apply SW trigger, combine overlay + dirt as MC 
mc = pd.concat([overlay.query('swtrig_pre==1'),dirt.query('swtrig_pre==1')], ignore_index=True)

In [ ]:
# separate by in/out FV & cosmic

infv = mc.query(in_fv_query+' and nu_purity_from_pfp>0.5')

In [ ]:
cosmic = mc.query(in_fv_query+' and nu_purity_from_pfp<=0.5')

In [ ]:
outfv = mc.query(out_fv_query)

In [ ]:

# check that everything is accounted for 
print(len(mc)==len(infv)+len(cosmic)+len(outfv))

if not (len(mc)==len(infv)+len(cosmic)+len(outfv)): 
    d = len(mc) - (len(infv)+len(cosmic)+len(outfv))
    print(d)
    
     
    m = pd.concat([infv, cosmic, outfv])
    diff = np.setdiff1d(list(mc.index),list(m.index))

    #for i in range(d):
        #print(mc.loc[diff[i], 'nu_purity_from_pfp'])
        #print(mc.loc[diff[i], 'nslice'])


In [ ]:
# how many signal events do we start with in FV -- normalized to overlay ? 

tot_signal_weighted = np.nansum(mc.query(signal+' and '+in_fv_query)['totweight_overlay'])
print(tot_signal_weighted)

# how many signal events are not cosmic cont?
tot_signal_weighted2 = np.nansum(mc.query('nu_purity_from_pfp>0.5'+' and '+signal+' and '+in_fv_query)['totweight_overlay'])
print(tot_signal_weighted2)


In [ ]:
# 5 main categories: 

# infv - overlay & dirt events with truth vtx in FV 
# outfv - overlay & dirt events with truth vtx in FV that are classified as neutrinos
# cosmic - overlay & dirt events with true vtx in FV that get misclassified as cosmic 
# ext - beam OFF data
# data - beam ON data 

datasets = [infv, outfv, cosmic, ext, data]

# Linear selection

In [ ]:
# apply preselection 
PRE_QUERY = 'nslice==1' # 
PRE_QUERY += ' and ' + reco_in_fv_query 
PRE_QUERY +=' and contained_fraction>0.9' 

PRE_QUERY += ' and n_showers_contained>0'
PRE_QUERY += ' and n_tracks_contained>0'


In [ ]:
# new optimized selection by Kaushal 
SEL_QUERY = PRE_QUERY
SEL_QUERY += ' and shr_energy_tot_cali>0.07' #

SEL_QUERY += ' and shr_score<0.125'
SEL_QUERY += ' and shrmoliereavg < 8'
SEL_QUERY += ' and trkpid<0'

SEL_QUERY += ' and n_showers_contained == 1'
SEL_QUERY += ' and shr_tkfit_dedx_Y<4'


if not ISRUN3: 
    SEL_QUERY += ' and tksh_distance<5'
    SEL_QUERY += ' and -0.9<tksh_angle<0.8'

else: 
    SEL_QUERY += ' and tksh_distance<4'
    SEL_QUERY += ' and -0.8<tksh_angle<0.8'

### data/MC comparisons

In [ ]:
# DATA / MC -- after nslice cut 

q = 'nslice==1'

plot_data('reco_nu_vtx_sce_x', 30, 0, 250, q,  datasets, ISRUN3, #ymax=25000,
          x_label="Reconstructed Interaction Vertex (X) [cm]")
plot_data('reco_nu_vtx_sce_y', 30, 0, 120, q,  datasets, ISRUN3, #ymax=20000,
          x_label="Reconstructed Interaction Vertex (Y) [cm]")
plot_data('reco_nu_vtx_sce_z', 30, 0, 1040, q,  datasets, ISRUN3, #ymax=17500,
          x_label="Reconstructed Interaction Vertex (z) [cm]")


In [ ]:
# after nslice & containment cuts
q += ' and '+reco_in_fv_query+ ' and contained_fraction>0.9'

plot_data('n_showers_contained', 5, 0, 5, q, datasets, ISRUN3, x_label="Number of Showers", ymax=15000)
plot_data('n_tracks_contained', 7, 0, 7, q, datasets, ISRUN3, x_label='Number of Tracks', ymax=15000)


In [ ]:
# after preselection 

q = PRE_QUERY

plot_data('shr_energy_tot_cali', 10, 0, 0.25, q, datasets, ISRUN3, 
          x_label='Total Calibrated Shower Energy [GeV]', ymax=2000) 

plot_data('CosmicIP', 10, 0, 50, q, datasets, ISRUN3, x_label='Distance to Closest Cosmic [cm]', ymax=500)
plot_data('hits_ratio', 20, 0, 1, q, datasets, ISRUN3, x_label='Shower Hits / Total Hits', ymax=800) 
plot_data('trkpid', 20, -1, 1, q, datasets, ISRUN3, x_label='Track PID Score') 
plot_data('shr_score', 15, 0, .5, q, datasets, ISRUN3, x_label='Pandora Leading Shower Score') 


In [ ]:
#tksh_angle
q = PRE_QUERY

plot_data('tksh_angle', 20, -1, 1, q, datasets, ISRUN3, x_label='cos($\\theta$)', ymax=800) 

In [ ]:
# after preselection + numu cuts

q = PRE_QUERY+' and shrmoliereavg<8 and trkpid<0 and shr_score<0.125'

p = plot_mc('shr_tkfit_dedx_Y', 20, 0, 6, q, datasets, ISRUN3, plt_norm='data', 
         x_label='dE/dx on the Collection Plane [MeV/cm]', ymax=75)

### Selection Performance

In [ ]:
# projected event distribution 

plot_mc('nu_e', 16, 0, 4, SEL_QUERY, datasets, ISRUN3, plt_norm='proj', pot='', ymax=110,
            x_label="True Neutrino Energy [GeV]")

In [ ]:
# list of cuts for selection performance plot 

cuts = ['nslice==1',  
        reco_in_fv_query, 
        'contained_fraction>0.9', 
        'n_tracks_contained>0', 
        'shr_energy_tot_cali>0.07', 
        'shr_score<0.125', 
        'shrmoliereavg<8', 
        'trkpid<0', 
        'n_showers_contained==1', 
        'shr_tkfit_dedx_Y<4']

if not ISRUN3: 
    cuts.append('tksh_distance<5')
    cuts.append('-0.9<tksh_angle<0.8')

else: 
    cuts.append('tksh_distance<4')
    cuts.append('-0.8<tksh_angle<0.8')
    
    
cut_names = ['Slice ID', "Reco'd in FV", 'Containment', "# Tracks", "Tot. Shr Energy", 
             'Shr Score', 'Shr Moliere Angle', 'Trk PID', '# Shrs', 'dE/dx (Y plane)', 
             'Trk/Shr Distance', 'Trk/Shr Angle']
    

In [ ]:
perf = sel_perf(cuts, datasets, 'totweight_overlay')

In [ ]:
plot_eff('nu_e', 16, 0, 4, SEL_QUERY, datasets, ISRUN3, x_label='True Neutrino Energy [GeV]', ymax=0.3)

In [ ]:
# purity & efficiency as a function of cut 

pur_cut = perf['purity (%)']*.01
eff_cut = perf['efficiency (%)']*.01
rel_eff_cut = perf['rel. eff. (%)']*.01

plt.figure(figsize=(10,7))

plt.plot(list(range(len(pur_cut))), pur_cut, '-o', color='firebrick', 
         label='Purity', markersize=3, linewidth=2)
plt.plot(list(range(len(pur_cut))), eff_cut, '-o', color='seagreen', 
         label='Efficiency', markersize=3, linewidth=2)
plt.plot(list(range(len(pur_cut))), rel_eff_cut, '-o', color='goldenrod', 
         label='Relative Efficiency', markersize=3, linewidth=2)

plt.grid(linestyle=":")

plt.xticks(list(range(12)), cut_names, ha='right', fontsize=12)
plt.xticks(rotation=40)
plt.yticks([0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1],  fontsize=12)

plt.legend(bbox_to_anchor=(0.32, 0.37), fontsize=15)

plt.title("Selection Performance", fontsize=20)
plt.tight_layout()
plt.show()

# BDT selection 

In [ ]:
BDT_PRE_QUERY = 'nslice==1'
BDT_PRE_QUERY += ' and ' + reco_in_fv_query
BDT_PRE_QUERY +=' and contained_fraction>0.9'
BDT_PRE_QUERY += ' and n_tracks_contained>0'
BDT_PRE_QUERY += ' and n_showers_contained==1'
BDT_PRE_QUERY += ' and shr_energy_tot_cali>0.07'

BDT_LOOSE_CUTS = BDT_PRE_QUERY
BDT_LOOSE_CUTS += ' and shr_score<0.3'
BDT_LOOSE_CUTS += ' and trkpid<0.35'
BDT_LOOSE_CUTS += ' and shrmoliereavg<15'
BDT_LOOSE_CUTS += ' and shr_tkfit_dedx_Y<7'
BDT_LOOSE_CUTS += ' and tksh_distance<12'

In [ ]:
# pre-determined boosting round numbers
lc_rounds = 200

# test train split 
split = 0.5


In [ ]:
# BDT training 
bdt_lc = main_BDT(datasets, BDT_LOOSE_CUTS, lc_rounds, test_size=split)

In [ ]:
results_df = bdt_lc[0]
bdt_model = bdt_lc[1]
train_df = bdt_lc[2]
test_df = bdt_lc[3]

In [ ]:
# in case we want to save 
# bdt_model.save_model("bdt_model_feb2021.model")

In [ ]:
# split events into different categories 
datasets_bdt = split_events(results_df)

In [ ]:
# plot_mc --> BDT score
plot_mc('BDT_score', 20, 0, 1, '', datasets_bdt, ISRUN3, plt_norm='proj', 
        pot='9.23E20', ymax=250)


In [ ]:
# plot neutrino energy with systematics 
plot_mc('nu_e', [x/4 for x in range(21)], 0, 5, 'BDT_score>0.575', datasets_bdt, ISRUN3, 
        plt_norm='proj', pot='9.23E20', ymax=140, sys=True)

In [ ]:
# performance of the linear selection for comparison 

gen_num = sum(test_df.query('is_signal==1 or is_cont_signal==1')['totweight_overlay'])
eff_box = sum(test_df.query(SEL_QUERY+' and is_signal==1')['totweight_overlay'])/gen_num * 100

tot_sel = np.nansum(test_df.query(SEL_QUERY)['weight']) # EXT does not have totweight_overlay 
pur_box = sum(test_df.query(SEL_QUERY+' and is_signal==1')['totweight_overlay']) / tot_sel * 100

results_box = [pur_box, eff_box]

# with stat errors on the linear performance 
e = eff_box/100

eff_err = math.sqrt( (e*(1-e)) / gen_num ) * 100
pur_err = math.sqrt(sum(test_df.query(SEL_QUERY+' and is_signal==1')['totweight_overlay'])) / tot_sel * 100

results_box_err = [pur_err, eff_err]

In [ ]:
# x values
x = np.arange(0, 0.8, 0.025)

In [ ]:
# returns purity, purErr, eff, effErr
perf = bdt_pe(results_df, x, test_df)


In [ ]:
bdt_box_plot(perf, results_box, x, ISRUN3, save=False, results_box_err=results_box_err)

In [ ]:
# NOW, apply the model to real data for data/MC comparisons

varlist = [
    "shr_score", "shrmoliereavg", "trkpid",
    "n_showers_contained", "shr_tkfit_dedx_Y", "tksh_distance",
    "tksh_angle", "subcluster", "trkshrhitdist2"]

# apply cuts
data_bdt = data.copy()
data_bdt = data_bdt.query(BDT_LOOSE_CUTS)

# clean datasets 
for column in varlist:
        data_bdt.loc[(data_bdt[column] < -1.0e37) | (data_bdt[column] > 1.0e37), column] = np.nan
    
# create testing dmatrix 
data_test = xgb.DMatrix(data=data_bdt[varlist])
    
# apply the bdt selection
preds = bdt_model.predict(data_test)

# add columns for plotting 
data_bdt['BDT_score'] = preds


In [ ]:
datasets_bdt.append(data_bdt)

In [ ]:
plot_data('BDT_score', [x/10 for x in range(11)], 0, 1, '', datasets_bdt, ISRUN3, 
          plt_norm='pot', ymax=150, x_label='BDT Score') 


### BDT data/MC Comparisons

In [ ]:
# after SW trigger & nslice cuts

q = 'nslice==1'

plot_data('reco_nu_vtx_sce_x', 30, 0, 250, q,  datasets, ISRUN3, #ymax=25000,
          x_label="Reconstructed Interaction Vertex (X) [cm]", save=True, save_label='fhc_data')
plot_data('reco_nu_vtx_sce_y', 30, 0, 120, q,  datasets, ISRUN3, #ymax=20000,
          x_label="Reconstructed Interaction Vertex (Y) [cm]")
plot_data('reco_nu_vtx_sce_z', 30, 0, 1040, q,  datasets, ISRUN3, #ymax=17500,
          x_label="Reconstructed Interaction Vertex (z) [cm]")



In [ ]:
# after SW trigger, slice ID, and FV (quality cuts)

# after nslice & containment cuts
q += ' and '+reco_in_fv_query+ ' and contained_fraction>0.9'

plot_data('n_showers_contained', 5, 0, 5, q, datasets, ISRUN3, x_label="Number of Showers", ymax=15000)
plot_data('n_tracks_contained', 7, 0, 7, q, datasets, ISRUN3, x_label='Number of Tracks', ymax=13000)
plot_data('shr_energy_tot_cali', 10, 0, 0.25, q, datasets, ISRUN3, x_label='Total Calibrated Shower Energy [GeV]', ymax=16000) 

In [ ]:
# after BDT preselection 

#shower score 
#trk PID
#shrmoliereavg 
#shr_tkfit_dedx_Y
#tksh_distance

q = BDT_PRE_QUERY

plot_data('shr_score', 10, 0, .5, q, datasets, ISRUN3, x_label='Pandora Leading Shower Score', ymax=1200) 
plot_data('trkpid', 10, -1, 1, q, datasets, ISRUN3, x_label='Track PID Score', ymax=800) 

plot_data('shrmoliereavg', 8, 0, 40, q, datasets, ISRUN3, x_label='Avg Shower Moliere Angle [degrees]', 
          ymax=700)

plot_data('shr_tkfit_dedx_Y', 10, 0, 10, q, datasets, ISRUN3, 
          x_label='dE/dx on the Collection Plane [MeV/cm]', ymax=500) 


In [ ]:
# after BDT Loose cuts

#shr_score
#shrmoliereavg 
#trk pid
#n_showers_contained
#subcluster
#trkshrhitdist2

q = BDT_LOOSE_CUTS


plot_data('shr_score', 10, 0, .3, q, datasets, ISRUN3, x_label='Pandora Leading Shower Score', ymax=300) 

plot_data('shrmoliereavg', 5, 0, 15, q, datasets, ISRUN3, x_label='Avg Shower Moliere Angle [degrees]', 
        ymax=175)

plot_data('trkpid', 10, -1, .35, q, datasets, ISRUN3, x_label='Track PID Score', ymax=130) 



In [ ]:
# after BDT selection - now switch to datasets_BDT dataframes 

plot_data('BDT_score', 11, 0, 1, '', datasets_bdt, ISRUN3, plt_norm='pot', ymax=120, 
         save=False, save_label='fhc_data_BDT', sys=False) 
